<a href="https://colab.research.google.com/github/flexinJ/flexinBot/blob/main/flexinBotImprovements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading required packages
!pip install yfinance
!pip install numpy
!pip install pandas
!pip install hurst

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import of the necessary modules
import numpy as np
import pandas as pd
import math
import yfinance as yf
from hurst import compute_Hc
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from time import sleep

In [3]:
######################## Welcome ##########################
# Based in fibo sequence.
# Flexin (@moraisjota) make this ~ enjoy xD
######################## FlexinBot ########################

In [25]:
class tradingSystem():
  def flexin(symbol, dateRange, timeFrame):
    ######################## DataFrame ########################
    # Finance database import
    data = yf.Ticker(symbol).history(period=dateRange, interval=timeFrame)
    data.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
    ######################## Indicators ########################
    # Moving average
    # 8, 13 and 55 periods (EMA, Adj close)
    data['mA8'] = data['Close'].ewm(span=8, min_periods=8).mean()
    data['mA13'] = data['Close'].ewm(span=13, min_periods=13).mean()
    data['mA55'] = data['Close'].ewm(span=55, min_periods=55).mean()
    # Ichimoku Cloud
    # 8, 21 and 55 periods
    data['tenkanSen'] = (data['High'].rolling(window=8).max()+data['High'].rolling(window=8).min())/2
    data['kijunSen'] = (data['High'].rolling(window=21).max()+data['Low'].rolling(window=21).min())/2
    data['senkouA'] = ((data['tenkanSen']+data['kijunSen'])/2).shift(21)
    data['senkouB'] = ((data['High'].rolling(window=55).max()+data['Low'].rolling(window=55).min())/2).shift(21)
    # Relative Strength Index (RSI)
    # 21 periods (close) 
    diff = data['Close'].diff()
    diffUp = diff.clip(lower=0)
    diffDown = -1*diff.clip(upper=0)
    averageGain = diffUp.ewm(span=21, min_periods=21).mean()
    averageLoss = abs(diffDown.ewm(span=21, min_periods=21).mean())
    relativeStrength = averageGain / averageLoss
    data['RSI'] = RSI = 100.0 - (100.0 / (1.0 + relativeStrength))
    # Calculate series of Log return of close prices 
    data['logCloseReturn'] = np.log(data['Close']/data['Close'].shift(1))
    ######################## Hurst exponent filter ########################
    '''
    H = 0.5 — Brownian motion
    0.5 < H < 1.0 — persistent behavior
    0 < H < 0.5 — anti-persistent behavior
    '''
    # Funcion def calculate Hurst Exponent (available on github on https://github.com/Mottl/hurst)
    H = lambda x: compute_Hc(x)[0]
    data['Hurst'] = data['logCloseReturn'].rolling(window=100).apply(H)
    # Dealing with NaN data
    data.dropna(inplace=True, axis=0)
    ######################## Current values ########################
    # Capturing all required current values of variables
    currentValues = dict()
    # Close price
    currentValues['Close price'] = currentClosePrice = data['Close'][-1]
    # Open price
    currentValues['Open price'] = currentOpenPrice = data['Open'][-1]
    # Moving average
    currentValues['Ma8'] = currentMa8 = data['mA8'][-1]
    currentValues['Ma13'] = currentMa13 = data['mA13'][-1]
    currentValues['Ma55'] = currentMa55 = data['mA55'][-1]
    # Ichimoku Cloud
    currentValues['Senkou A'] = currentSenkouA = data['senkouA'][-1]
    currentValues['Senkou B'] = currentSenkouB = data['senkouB'][-1]
    # RSI
    currentValues['Rsi'] = currentRsi = data['RSI'][-1]
    # Hurst exponent
    currentValues['Hurst'] = currentHurst = data['Hurst'][-1]
    print('-----------------------------------------------------------------')
    # Plot hurst graph
    '''data['Hurst'].tail(5).plot(figsize=(7.7, 5), color='deepskyblue', linestyle='dashdot').grid(True)
    plt.title(f'Hurst Exponent of {symbol}', fontsize=20)
    plt.xlabel('Time', size=12,  color = 'r')
    plt.ylabel('Hurst value', size=12, color = 'deepskyblue') 
    plt.show()'''
    # View current data
    print('####################### Flexin strategy 🤖 ######################')
    currentValues = dict(sorted(currentValues.items(), key=lambda item: item[1]))
    for k, v in currentValues.items():
      currentValues = sorted(currentValues)
      print(f'{k}| {v.round(2)}')
    print('######################## Flexin signals! ########################')
    ######################## Flexin strategy ########################
    if 1 > currentHurst > 0.55:
      print(f'WOW! Hurst exponent is equal to {currentHurst.round(2)} w/ persistent behavior.')
      # Buy signal
      if currentRsi > 50:
        print(f'Relative strength index above 50% level. ⬆️')
        if currentSenkouA > currentSenkouB:
          print(f'Floating above the clouds. 😶‍🌫️')
          if currentClosePrice > currentMa8 and currentClosePrice > currentMa13 and currentClosePrice > currentMa55:
            buy = True
            print(f'Uptrend probability at {currentClosePrice.round(2)}, sr. J!!! 📈📈📈')
            while True in buy:
              print('Buy order is open.')
              if currentRsi < 50 and currentClosePrice < currentMa8 and currentClosePrice < currentMa13 and currentClosePrice < currentMa55:
                print('Close buy order.')
                buy = False
      else:
        print(f'Relative strength index below 50% level ⬇️')
        if currentSenkouA < currentSenkouB:
          print(f'Floating below the clouds. 🌥️')
          if currentClosePrice < currentMa8 and currentClosePrice < currentMa13 and currentClosePrice < currentMa55:
            sell = True
            print(f'Downtrend probability at {currentClosePrice.round(2)}, sr. J!!! 📉📉📉')
            while True in sell:
              print('Sell order is open.')
              if currentRsi < 50 and currentClosePrice > currentMa8 and currentClosePrice > currentMa13 and currentClosePrice > currentMa55:
                print('Close sell order.')
                sell = False
    else:
      print('Wait...! Anti-persistent behavior.')
    '''# Buy
    if currentClosePrice > currentMa8 and currentOpenPrice > currentMa8 and currentMa8 > currentMa13 and currentMa13 > currentMa55 and currentMa55 > currentSenkouA and currentSenkouA > currentSenkouB and currentRsi > 50 and 1 > currentHurst > 0.55:
      buy = True
      print(f'Buy: order {buy} at {currentClosePrice}')
    else:
      print('Buy: False')
    # Sell
    if currentClosePrice < currentMa8 and currentOpenPrice < currentMa8 and currentMa8 < currentMa13 and currentMa13 < currentMa55 and currentMa55 < currentSenkouA and currentSenkouA < currentSenkouB and currentRsi < 50 and 1 > currentHurst > 0.55:
      sell = True
      print(f'Buy: order {sell} at {currentClosePrice}')
    else:
      print('Sell: False')'''

In [ ]:
while True:
  tradingSystem.flexin('^HSI', '5d', '1m')
  sleep(60)
  clear_output()

-----------------------------------------------------------------
####################### Flexin strategy 🤖 ######################
Hurst| 0.2
Rsi| 20.67
Close price| 18685.37
Open price| 18685.37
Ma8| 18687.75
Ma13| 18696.38
Ma55| 18777.53
Senkou A| 18839.31
Senkou B| 18853.81
######################## Flexin signals! ########################
Wait...! Anti-persistent behavior.
